In [45]:
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from sympy import *
from math import factorial
import platform
import time

In [46]:
def extractStrokes(fileName: str):
    with open(fileName) as f:
        data = json.load(f)
    totalStrokes = len(data['strokes'])
    strokes = []
    for i in range(totalStrokes):
        if data['strokes'][i]['draw_type'] == 0:
            strokes.append(data['strokes'][i]['path'])
        else:
            continue
    #print("Total strokes: ", totalStrokes)
    return strokes

In [47]:
def comb(n, r):
    if r == 0:
        return 1
    else:
        return factorial(n) // (factorial(r) * factorial(n - r))

In [48]:
def extractCoeff(num):
    x = Symbol('x')
    b0, b1, b2, b3, b4, b5 = symbols('b0 b1 b2 b3 b4 b5')
    l = [b0, b1, b2, b3, b4, b5]
    m = 0
    for i in range(num):
        m += comb(num-1, i) * (x ** i) * ((1 - x) ** (num-1 - i)) * l[i] 
    coef = Poly(m, x).coeffs()
    coef.reverse()
    coef = [[coef[i].coeff(l[j]) for j in range(num)] for i in range(num)]
    return np.array(coef, dtype=np.float64)

In [49]:
def getControlPoints(points, order):
    M = extractCoeff(order)
    P = points
    length = len(P)
    D = [0]
    for i in range(1, length):
        D.append(D[i-1] + np.linalg.norm(P[i] - P[i-1]))   
    S = [D[i] / D[-1] for i in range(length)]
    S = np.array(S)
    '''if np.isnan(S).any():
        print(points)'''
    T = np.tile(S, (order, 1)).transpose()
    T = np.power(T, np.arange(order))
    C = np.matmul(np.matmul(np.matmul(np.linalg.inv(M), np.linalg.pinv(np.matmul(T.transpose(), T))), T.transpose()), P)
    #print(C)
    return C



In [50]:
def generatePath(points, order, T):
    M = extractCoeff(order)
    C = getControlPoints(points, order)
    #print(T.shape, M.shape, C.shape)
    path = np.matmul(np.matmul(T, M), C)
    return path

In [51]:
def getRange(ran, p):
    pmax = np.max(p, axis=0)
    pmin = np.min(p, axis=0)
    #print(pmax, pmin)
    if ran[0][0] > pmin[0]:
        ran[0][0] = pmin[0]
    if ran[0][1] > pmin[1]:
        ran[0][1] = pmin[1]
    if ran[1][0] < pmax[0]:
        ran[1][0] = pmax[0]
    if ran[1][1] < pmax[1]:
        ran[1][1] = pmax[1]
    return ran

In [52]:
def removeDuplicate(points):
    newPoints = []
    for i in range(len(points)):
        if i == 0:
            newPoints.append(points[i])
        else:
            if np.linalg.norm(points[i] - points[i-1]) > 0:
                newPoints.append(points[i])
    return np.array(newPoints)

In [53]:
def drawStrokes(strokes, t, file, imgFile):
    plt.figure(figsize=(10, 5))
    pic = plt.imread(imgFile)
    ax1 = plt.subplot(121)
    ax1.imshow(pic)
    ax2 = plt.subplot(122)
    #ran = [[100, 100], [0, 0]]
    for points in strokes:
        points = np.array(points)
        if np.unique(points, axis=0).shape[0] == 1:
            ax2.plot(points[:,0], points[:,1], 'bo', markersize=0.1)
            continue
        if len(points) <= 6:
            order = len(points) - 1
        else:
            order = 6
        T = np.tile(t, (order, 1)).transpose()
        T = np.power(T, np.arange(order))
        points = removeDuplicate(points)
        if len(points) > 20:
            for i in range(0, len(points), 19):
                p = points[i:i+20]
                if np.unique(p, axis=0).shape[0] == 1:
                    continue
                #ran = getRange(ran, p)
                path = generatePath(p, order, T)
                px, py = path[:,0], path[:,1]
                ax2.plot(px, py, 'b-')
        else:
            #ran = getRange(ran, points)
            path = generatePath(points, order, T)
            px, py = path[:,0], path[:,1]
            ax2.plot(px, py, 'b-')
    ax2.axis('equal')
    ax2.invert_yaxis()
    savePath = 'curvefittingimg/'+ file[:-5] + '_generated.png'
    plt.savefig(savePath)
     
    #print('X: ', ran[0][0],'-',ran[1][0], '\nY: ', ran[0][1],'-',ran[1][1])


In [54]:
def drawStrokes(strokes, t, file, saveDir):
    #plt.figure(figsize=(10, 10))
    reduceP = [1, 2, 4 ,8]
    #ran = [[100, 100], [0, 0]]
    for r in reduceP:
        rr = 800 / r
        CPrecord = {'control_points':[]} #control points record
        for points in strokes:
            strokeCP = {}
            CPid = 1
            if r == 1:   
                points = np.array(points)
            else:
                points = np.rint(np.array(points) / r) 
            if np.unique(points, axis=0).shape[0] == 1:
                #plt.plot(points[:1,0], points[:1,1], 'bo', markersize=0.1)
                strokeCP[CPid] = points[0].tolist()
                CPrecord['control_points'].append(strokeCP)
                continue
            if len(points) <= 6:
                order = len(points) - 1
            else:
                order = 6
            T = np.tile(t, (order, 1)).transpose()
            T = np.power(T, np.arange(order))
            points = removeDuplicate(points)
            if len(points) > 20:
                for i in range(0, len(points), 19):
                    p = points[i:i+20]
                    if np.unique(p, axis=0).shape[0] == 1:
                        continue
                    #ran = getRange(ran, p)
                    C = getControlPoints(p, order)
                    strokeCP[CPid] = C.tolist()
                    CPid += 1
                    '''path = generatePath(p, order, T)
                    px, py = path[:,0], path[:,1]
                    plt.plot(px, py, 'b-')'''
            else:
                #ran = getRange(ran, points)
                C = getControlPoints(points, order)
                strokeCP[CPid] = C.tolist()
                CPid += 1
                '''path = generatePath(points, order, T)
                px, py = path[:,0], path[:,1]
                plt.plot(px, py, 'b-')'''
            CPrecord['control_points'].append(strokeCP)

        #print(CPrecord)
        with open('%s/%djson/%s.json' % (saveDir, rr, file[:-5]), 'w') as f:
            json.dump(CPrecord, f)
        '''plt.axis('equal')
        plt.gca().invert_yaxis()
        savePath = '%s/%dimg/%s_generated.png' % (saveDir, rr, file[:-5])
        plt.savefig(savePath)
        plt.clf()

    plt.close()'''
     
    #print('X: ', ran[0][0],'-',ran[1][0], '\nY: ', ran[0][1],'-',ran[1][1])


In [57]:
for category in ['Human_Face', 'Industrial_Component', 'Lamp', 'Primitive', 'Shoe', 'Vehicle']:
    dirPath = 'C:/Users/ssinc/Desktop/CityU/fyp/DifferSketching_Dataset/' + category + '/' + 'original_json'
    #originalImage = category + '\\' + 'original_png'
    dir_list = os.listdir(dirPath)
    os.makedirs('fitImg/%s' % category, exist_ok=True)
    for i in [100, 200, 400, 800]:
        #os.makedirs('fitImg/%s/%dimg' % (category, i), exist_ok=True)
        os.makedirs('fitImg/%s/%djson' % (category, i), exist_ok=True)
    #img_list = os.listdir(originalImage)
    saveDir = 'fitImg/%s' % category
    for i in range(len(dir_list)):
        fileName = dirPath + '/' + dir_list[i]
        #imgName = originalImage + '\\' + img_list[i]
        step = 0.001 #put the time step here

        strokes = extractStrokes(fileName)
        t = np.arange(0,1,step)
        #rang = [0, 0]

        #drawStrokes(strokes, t, dir_list[i], imgName)
        drawStrokes(strokes, t, dir_list[i], saveDir)
    
    
    